# Distributed Model Calibration

- please change the Path to the directory where you stored the case study data

In [1]:
import os


os.chdir("../../../")

the currect work directory should be at the root directory of the repository


In [4]:
import numpy as np
import datetime as dt
from osgeo import gdal
from Hapi.rrm.calibration import Calibration
import Hapi.rrm.hbv_bergestrom92 as HBV

from Hapi.rrm.distparameters import DistParameters as DP
import Hapi.sm.performancecriteria as PC

### Paths

In [5]:
Path = "Examples/Hydrological model/data/distributed_model/"
PrecPath = Path + "/prec"
Evap_Path = Path + "/evap"
TempPath = Path + "/temp"
FlowAccPath = Path + "/GIS/acc4000.tif"
FlowDPath = Path + "/GIS/fd4000.tif"
CalibPath = Path + "/calibration"
SaveTo = Path + "/results"

# Calibration Object

In [6]:
AreaCoeff = 1530
#[sp,sm,uz,lz,wc]
InitialCond = [0, 5, 5, 5, 0]
Snow = 0

"""
Create the model object and read the input data
"""
Sdate = '2009-01-01'
Edate = '2009-04-10'
name = "Coello"
Coello = Calibration(name, Sdate, Edate, SpatialResolution="Distributed")

### Meteorological & GIS Data

In [7]:
Coello.readRainfall(PrecPath)
Coello.readTemperature(TempPath)
Coello.readET(Evap_Path)

Coello.readFlowAcc(FlowAccPath)
Coello.readFlowDir(FlowDPath)
Coello.readLumpedModel(HBV, AreaCoeff, InitialCond)

2022-03-20 00:52:44.312 | DEBUG    | catchment:readRainfall:200 - Rainfall data are read successfully
2022-03-20 00:52:44.359 | DEBUG    | catchment:readTemperature:253 - Temperature data are read successfully
2022-03-20 00:52:44.406 | DEBUG    | catchment:readET:295 - Potential Evapotranspiration data are read successfully
2022-03-20 00:52:44.406 | DEBUG    | catchment:readFlowAcc:387 - Flow Accmulation input is read successfully
2022-03-20 00:52:44.422 | DEBUG    | catchment:readFlowDir:441 - Flow Direction input is read successfully
2022-03-20 00:52:44.422 | DEBUG    | catchment:readLumpedModel:678 - Lumped model is read successfully


In [8]:
UB = np.loadtxt(CalibPath + "/UB - tot.txt", usecols=0)
LB = np.loadtxt(CalibPath + "/LB - tot.txt", usecols=0)
Coello.readParametersBounds(UB, LB, Snow)

2022-03-20 00:52:46.674 | DEBUG    | catchment:readParametersBounds:914 - Parameters bounds are read successfully


### spatial variability function

define how generated parameters are going to be distributed spatially
totaly distributed or totally distributed with some parameters are lumped
for the whole catchment or HRUs or HRUs with some lumped parameters
for muskingum parameters k & x include the upper and lower bound in both
UB & LB with the order of Klb then kub
function inside the calibration algorithm is written as following

par_dist = SpatialVarFun(par,*SpatialVarArgs,kub=kub,klb=klb)


In [9]:
raster = gdal.Open(FlowAccPath)
#-------------
# for lumped catchment parameters
no_parameters = 12
klb = 0.5
kub = 1
#------------
no_lumped_par = 1
lumped_par_pos = [7]

SpatialVarFun = DP(raster, no_parameters, no_lumped_par=no_lumped_par,
                   lumped_par_pos=lumped_par_pos, Function=2, Klb=klb, Kub=kub)
# calculate no of parameters that optimization algorithm is going to generate
SpatialVarFun.ParametersNO

980

### Gauges

In [10]:
Coello.readGaugeTable(Path + "/stations/gauges.csv", FlowAccPath)
GaugesPath = Path + "/stations/"
Coello.readDischargeGauges(GaugesPath, column='id', fmt="%Y-%m-%d")

2022-03-20 00:52:52.225 | DEBUG    | catchment:readGaugeTable:773 - Gauge Table is read successfully
2022-03-20 00:52:52.824 | DEBUG    | catchment:readDischargeGauges:867 - Gauges data are read successfully


In [11]:
Coello.GaugesTable

,id,name,x,y,original area,area,area ratio,weight,cell_row,cell_col
0,1,Station 1,454795.6728,503143.3264,124.0659,64,1.938530,0.06,4.0,5.0
1,2,Station 2,443847.5736,481850.7151,83.0025,96,0.864609,0.08,9.0,2.0
2,3,station 3,454044.6935,481189.4256,44.8587,16,2.803669,0.02,9.0,5.0
3,4,Station 4,464533.7067,502683.6482,91.4949,80,1.143686,0.07,4.0,7.0
4,5,Station 5,463231.1242,486656.3455,730.1259,784,0.931283,0.38,8.0,7.0
5,6,ExitPoint_coello basin,487292.5152,478045.5720,1453.9185,1408,1.032613,0.40,10.0,13.0


### Objective function

In [12]:
coordinates = Coello.GaugesTable[['id', 'x', 'y', 'weight']][:]

# define the objective function and its arguments
OF_args = [coordinates]


def OF(Qobs, Qout, q_uz_routed, q_lz_trans, coordinates):
    Coello.extractDischarge()
    all_errors = []
    # error for all internal stations
    for i in range(len(coordinates)):
        all_errors.append((PC.RMSE(Qobs.loc[:, Qobs.columns[0]],
                                   Coello.Qsim[:, i])))  #*coordinates.loc[coordinates.index[i],'weight']
    print(all_errors)
    error = sum(all_errors)
    return error


Coello.readObjectiveFn(OF, OF_args)

Objective function is read successfully


### Optimization

API options
Create the options dictionary all the optimization parameters should be passed
to the optimization object inside the option dictionary:

to see all options import Optimizer class and check the documentation of the
method setOption

- for the filename please provide the full path


In [13]:
ApiObjArgs = dict(hms=100, hmcr=0.95, par=0.65, dbw=2000, fileout=1,
                  filename=SaveTo + "/Coello_" + str(dt.datetime.now())[0:10] + ".txt")

for i in range(len(ApiObjArgs)):
    print(list(ApiObjArgs.keys())[i], str(ApiObjArgs[list(ApiObjArgs.keys())[i]]))

#pll_type = 'POA'
pll_type = None

ApiSolveArgs = dict(store_sol=True, display_opts=True, store_hst=True, hot_start=False)

OptimizationArgs = [ApiObjArgs, pll_type, ApiSolveArgs]

hms 100
hmcr 0.95
par 0.65
dbw 2000
fileout 1
filename Examples/Hydrological model/data/distributed_model//results/Coello_2022-03-20.txt


### Run Calibration

In [ ]:
cal_parameters = Coello.runCalibration(SpatialVarFun, OptimizationArgs,
                                       printError=0)

Calibration starts

Optimization Problem -- HBV Calibration

        Objective Function: opt_fun

    Objectives:
        Name        Value        Optimum
	     f               0             0

	Variables (c - continuous, i - integer, d - discrete):
        Name    Type       Value       Lower Bound  Upper Bound
	     x0       c	      0.000000       7.00e-01     1.30e+00 
	     x1       c	      0.000000       5.00e+01     4.00e+02 
	     x2       c	      0.000000       1.15e+00     2.50e+00 
	     x3       c	      0.000000       1.00e-01     3.00e-01 
	     x4       c	      0.000000       4.60e-01     1.00e+00 
	     x5       c	      0.000000       2.27e-01     8.65e-01 
	     x6       c	      0.000000       1.24e-01     8.00e-01 
	     x7       c	      0.000000       1.61e+01     7.23e+01 
	     x8       c	      0.000000       1.66e+00     5.28e+00 
	     x9       c	      0.000000       1.00e+00     1.00e+02 
	    x10       c	      0.000000       2.00e-01     5.00e-01 
	    x11       

### Convert Parameters to rasters

In [ ]:
SpatialVarFun.Function(Coello.Parameters, kub=SpatialVarFun.Kub, klb=SpatialVarFun.Klb)
SpatialVarFun.saveParameters(SaveTo)